## 03. 깊은 CNN으로 MNIST 분류하기
---

In [20]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [22]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [23]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=False)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=False)

In [24]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last =True)

In [25]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5        # drop_out 용! 남길 비율

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        self.fc1 = torch.nn.Linear(4*4*128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1-self.keep_prob))

        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [26]:
model = CNN().to(device)

In [27]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [28]:
total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

총 배치의 수: 600


In [29]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] Cost = {:>.9}'.format(epoch+1, avg_cost))

[Epoch:    1] Cost = 0.210043013
[Epoch:    2] Cost = 0.0537214801
[Epoch:    3] Cost = 0.0389906354
[Epoch:    4] Cost = 0.0302699804
[Epoch:    5] Cost = 0.024625374
[Epoch:    6] Cost = 0.0187018253
[Epoch:    7] Cost = 0.0169688836
[Epoch:    8] Cost = 0.0155189531
[Epoch:    9] Cost = 0.0140519328
[Epoch:   10] Cost = 0.0121285645
[Epoch:   11] Cost = 0.0111270649
[Epoch:   12] Cost = 0.0093757147
[Epoch:   13] Cost = 0.00850844756
[Epoch:   14] Cost = 0.00823599566
[Epoch:   15] Cost = 0.00919211376


In [30]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

C:\Users\lemon\anaconda3\envs\chch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\lemon\anaconda3\envs\chch\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9833999872207642
